In [1]:
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import string
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import plot_confusion_matrix, classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
import seaborn as sns

stopwords = stopwords.words('english')

In [2]:
df = pd.read_csv('data/IMDB Dataset.csv')
print(df.shape)
df.head()

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
print(df.isnull().sum())
print(df.duplicated().sum())

review       0
sentiment    0
dtype: int64
418


In [4]:
df.columns

Index(['review', 'sentiment'], dtype='object')

In [5]:
df.drop_duplicates(subset='review', inplace=True)

In [6]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

Feature Engineering

In [7]:
def preprocess(str):
    str = str.lower()
    tk = RegexpTokenizer(r'[a-zA-Z]+')
    tokens = [token for token in tk.tokenize(str)]
    tokens = [token for token in tokens if token not in stopwords]
    porter = PorterStemmer()
    tokens = [porter.stem(token) for token in tokens]
    
    return ' '.join(tokens)

In [8]:
df['clean_review'] = df['review'].apply(preprocess)
df.drop(['review'], axis=1, inplace=True)

In [9]:
df.head()

,sentiment,clean_review
0,positive,one review mention watch oz episod hook right ...
1,positive,wonder littl product br br film techniqu unass...
2,positive,thought wonder way spend time hot summer weeke...
3,negative,basic famili littl boy jake think zombi closet...
4,positive,petter mattei love time money visual stun film...


In [10]:
df['clean_review'][0]

'one review mention watch oz episod hook right exactli happen br br first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use word br br call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home mani aryan muslim gangsta latino christian italian irish scuffl death stare dodgi deal shadi agreement never far away br br would say main appeal show due fact goe show dare forget pretti pictur paint mainstream audienc forget charm forget romanc oz mess around first episod ever saw struck nasti surreal say readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard sold nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch oz may becom comfort uncomfort view that get touch da

In [11]:
df = pd.get_dummies(df, columns=['sentiment'], drop_first=True)

In [12]:
X = df['clean_review']
y = df['sentiment_positive']

In [13]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))

In [14]:
X = vectorizer.fit_transform(X)

In [15]:
X.shape

(49582, 2626705)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

- Random Forest Classifier

In [17]:
rf_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=11)

rf_clf.fit(X_train, y_train)

RandomForestClassifier(max_leaf_nodes=16, n_estimators=500, n_jobs=11)

In [18]:
rf_clf.score(X_test, y_test)

0.8404759503882222

In [ ]:
new_review = 'petter mattei love time money visual stun film watch mr mattei offer us vivid portrait human relat movi seem tell us money power success peopl differ situat encount br br variat arthur schnitzler play theme director transfer action present time new york differ charact meet connect one connect one way anoth next person one seem know previou point contact stylishli film sophist luxuri look taken see peopl live world live habitat br br thing one get soul pictur differ stage loneli one inhabit big citi exactli best place human relat find sincer fulfil one discern case peopl encount br br act good mr mattei direct steve buscemi rosario dawson carol kane michael imperioli adrian grenier rest talent cast make charact come aliv br br wish mr mattei good luck await anxious next work'
#new_review = preprocess(new_review)
print(new_review)

In [ ]:
new_vectorized = vectorizer.transform([new_review])
print(new_vectorized)

In [ ]:
y_pred = rf_clf.predict(new_vectorized)
print(y_pred)

In [ ]:
pp = rf_clf.predict_proba(new_vectorized)
print(pp)

In [ ]:
rf_clf.feature_importances_

- MultinomialNB

In [19]:
model = MultinomialNB(alpha = 0.06)
model.fit(X_train, y_train)
accuracy = model.score(X_test, y_test)

In [20]:
print(accuracy)

0.8826257940909549
